<a href="https://colab.research.google.com/github/rafajak/gpt3_examples/blob/master/logit_bias_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install transformers

import json
from google.colab import files
import openai
from transformers import GPT2Tokenizer
import pandas as pd

     |████████████████████████████████| 163kB 2.6MB/s 
  Created wheel for openai: filename=openai-0.2.4-cp36-none-any.whl size=170710 sha256=ce805e2107bee6b4f2b57f2fcf0de7cc289da125997e6e8faee3da3445571cc0
  Stored in directory: /root/.cache/pip/wheels/74/96/c8/c6e170929c276b836613e1b9985343b501fe455e53d85e7d48
Successfully built openai
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 17.2MB/s 
     |████████████████████████████████| 1.1MB 38.8MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1eec481a6dd064eb7bd276504c178cb5247ec459227d637ac18555779f24df34
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
# To keep your secret API key secure - create a api_key.json file containing the key and upload it with the function below. 
# My config file looks like this: {"api_key": "heresmysecretkey"}
upload = files.upload()
openai.api_key = json.load(open("api_key.json", "r"))["api_key"]

Saving api_key.json to api_key.json


In [3]:
kwargs = {"engine": "curie",
          "temperature": 0,
          "max_tokens": 20,
          "stop": "\n",
          "logprobs": 5,
          "echo": True}

prompt = "The quick brown fox jumps over the lazy" # --> the next token will be ' dog'

In [4]:
r =  openai.Completion.create(prompt = prompt, **kwargs) 

pd.DataFrame(r["choices"][0])["text"]["tokens"]

'The quick brown fox jumps over the lazy dog.'

In [5]:
# Suppose we don't want the prompt to be completed with 'dog'.
word_to_block = " dog"

#Initialize the BPE tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') 

#Get an token id for the token we want to penalize
token_id_to_block = tokenizer.encode(word_to_block)[0]

kwargs["logit_bias"] = {token_id_to_block: -100}

r =  openai.Completion.create(prompt = prompt, **kwargs) 

pd.DataFrame(r["choices"][0])["text"]["tokens"]

'The quick brown fox jumps over the lazy brown bear.'